In [1]:
import os
import sys

sys.path.append('../multitask-counter-arg-generation/src-py/')
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [2]:
import csv
import random
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import torch
import transformers
from rouge_score import rouge_scorer
from transformers import BartTokenizer, BartForConditionalGeneration
from torch.nn import CrossEntropyLoss, MSELoss
import os
from model import MultiTaskBart
from model import OurModel
from utils import parse_df
import time
import sys
import tqdm
import pandas as pd
from datasets import load_dataset, load_metric, Dataset

import nltk
pd.set_option('display.max_colwidth', None)

In [20]:
import prompted_conclusion_utils as conc_utils
from prompted_conclusion_utils import *

from transformers.generation_logits_process import * 

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using GPU? ", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0))


data_dir = '../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/'
teacher_model_path='../multitask-counter-arg-generation/data/output/stance_classification/best_model/'

batch_size=4

Using GPU?  True
Device name: NVIDIA A100-SXM4-40GB


In [5]:
#Teacher model
stance_classifier_teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_path)
stance_classifier_teacher_model     = AutoModelForSequenceClassification.from_pretrained(teacher_model_path)
arg_stance_pipeline = TextClassificationPipeline(model=stance_classifier_teacher_model, tokenizer=stance_classifier_teacher_tokenizer, framework='pt', task='stance_classification', device=0)

In [6]:
#Our model versions with the best performing checkpoint on the validation set..
model_without_stance    = OurModel.load('../multitask-counter-arg-generation/data/output/ca-final-models/mt-v4.baseline_1/trained_models/models-global-step-5500', 'facebook/bart-large',  model_config=transformers.AutoConfig.from_pretrained('facebook/bart-large'))
model_with_stance = OurModel.load('../multitask-counter-arg-generation/data/output/ca-final-models/mt-v4.baseline_2/trained_models/models-global-step-4000', 'facebook/bart-large',  model_config=transformers.AutoConfig.from_pretrained('facebook/bart-large'))

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained('facebook/bart-large')

model_without_stance.to(device)
model_with_stance.to(device)

_ = model_with_stance.eval()
_ = model_without_stance.eval()

In [21]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def get_stance_scores(sents1, sents2):
    #compute stance score using our trained model
    text_inputs = [x[0] + ' </s> ' + x[1] for x in zip(sents1, sents2)]
    stance_results = arg_stance_pipeline(text_inputs, truncation=True)
    stance_labels = [int(x['label'].split('_')[-1]) for x in stance_results]
    stance_scores = [x['score'] for x in stance_results]
    return sum(stance_labels)/len(stance_labels), stance_labels, stance_scores  #The score is the percentage of cases we generated a counter

def counters_coherence(post_conclusions, post_counters):
    post_counters = [nltk.sent_tokenize(x) for x in post_counters]
    conclusion_counter_sent_pairs = [(x[1], s) for x in zip(post_counters, post_conclusions) for s in x[0]]
    #print(conclusion_counter_sent_pairs)
    conclusions, counter_sents = zip(*conclusion_counter_sent_pairs)
    _, stance_labels, stance_scores = get_stance_scores(conclusions, counter_sents)
    stance_scores = [x[0] * -1 if x[1] == 0 else x[0] for x in zip(stance_scores, stance_labels)]

    #collect counter_scores
    counter_scores = []
    idx = 0
    #print(len(stance_scores))
    for i, post_counter in enumerate(post_counters):
        #print(len(post_counter))
        counter_scores.append(stance_scores[idx: idx + len(post_counter)])
        idx+=len(post_counter)
    
    #print(counter_scores)
    return [np.mean(s) for s in counter_scores]

def get_best_counters(conclusions, counters):
    #choose best counter
    best_counters = []
    for post_conclusions, post_counters  in zip(conclusions, counters):
        scores = counters_coherence(post_conclusions, post_counters)
        best_counters.append((post_conclusions[np.argmax(scores)], post_counters[np.argmax(scores)]))
        
    return best_counters

def generate_multiple_counters(model, tokenizer, df, conclusion_gen_kwargs, argument_gen_kwargs, 
                               output_clm='post_counters', post_concepts_clm='post_concepts', num_candidates=6, batch_size=8):
    
    post_counters = []
    for i in range(num_candidates):
        control_codes = [x[i] if i < len(x) else '' for x in df[post_concepts_clm].tolist()]
        premises = df['post'].tolist()
        
        print(control_codes[:3])
        conclusions, counters   = generate_two_seq_prompted_counters(model, tokenizer, premises, 
                                                                     control_codes, conclusion_gen_kwargs, argument_gen_kwargs, 
                                                                     batch_size=batch_size)
                
        post_counters.append(zip(control_codes, conclusions, counters))

    df[output_clm] = list(map(list, zip(*post_counters))) # transpose

    return df


def generate_counters(model, tokenizer, data_loader, argument_gen_kwargs, conclusion_gen_kwargs, skip_special_tokens=True):
    processors = LogitsProcessorList()
    generated_counter_arguments = []
    generated_conclusions = []
    model.eval()
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            generated_argument_tokens   = model.generate_counter_argument(input_ids, attention_mask, argument_gen_kwargs, processors)
            generated_conclusion_tokens = model.generate_conclusion(input_ids, attention_mask, conclusion_gen_kwargs, processors)
                        
            generated_argument_tokens = generated_argument_tokens.cpu().numpy()
            decoded_arguments = tokenizer.batch_decode(generated_argument_tokens, skip_special_tokens=skip_special_tokens)
            
            generated_conclusion_tokens = generated_conclusion_tokens.cpu().numpy()
            decoded_conclusions = tokenizer.batch_decode(generated_conclusion_tokens, skip_special_tokens=skip_special_tokens)
            
            generated_counter_arguments +=decoded_arguments
            generated_conclusions +=decoded_conclusions
            
    return generated_conclusions, generated_counter_arguments

In [9]:
conclusion_gen_kwargs = {
    "do_sample": True, 
    "max_length":50,
    "top_p":0.95, 
    "num_beams":1,
}

argument_gen_kwargs = {
    "do_sample": True, 
    "max_length":100,
    "min_length":50,
    "top_p":0.95, 
    "no_repeat_ngram_size":3,
    "top_k": 50,
    "num_beams":1,
}

In [10]:
#df = pd.read_pickle(data_dir + 'test_conclusion_all_preprocessed.pkl')[['post_id', 'title', 'post', 'counter']]
df = pd.read_pickle('../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed_multi_conclusions.pkl')[['post_id', 'title', 'post_concepts2', 'post', 'counter']]
df['post'] = df.post.apply(lambda x: ' '.join(x))

In [11]:
df.head()

post_id  \
500025  t3_c92t5g   
534021  t3_d3cv11   
415645  t3_a50c29   
507298  t3_cetf5h   
505763  t3_cdupvi   

                                                                                                                                    title  \
500025                                                                                                    I Don't Believe in the Big Bang   
534021                                                                                There is no reason for Britain to remain in the EU.   
415645   The Trump Administration Would Be Significantly More Harmful to US Domestic and International Policy W/out Moderates Like Mattis   
507298                             Poisoning the well against fascists is a dishonest and therefore ineffective way of combating fascists   
505763                                                                       Women have an easier time finding romantic partners than men   

                                                                                                                                                                      post_concepts2  \
500025                                                                                                                           [Science, Religion, Fact, Idiot, Parking violation]   
534021                                                                                                [Law, Sovereignty, European Union law, Border, People, Idea, Trade bloc, Sign]   
415645  [Military, Transgender, Surgery, Torture, Headquarters, United States Secretary of Defense, Presidency of Barack Obama, Citizenship of the United States, Senate, Secretary]   
507298                                                 [Fascism, Nationalism, White supremacy, Video, Videotelephony, YouTuber, Sophist, Validity (logic), Dishonesty, ContraPoints]   
505763                                                                           [Female, Western world, Society, Time, Woman, Youth, Industry (economics), Belief, Friendship, Man]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
df = generate_multiple_counters(model_without_stance, 
                                                   tokenizer, df, conclusion_gen_kwargs, argument_gen_kwargs, 
                                                   output_clm='post_counters', 
                                                   post_concepts_clm='post_concepts2', 
                                                   num_candidates=10, 
                                                   batch_size=16)

['Science', 'Law', 'Military']


Parameter 'function'=<function generate_two_seq_prompted_counters.<locals>.<lambda> at 0x7f8d4c1ce940> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

100%|██████████| 125/125 [05:02<00:00,  2.42s/it]

['Religion', 'Sovereignty', 'Transgender']


...

  0%|          | 0/2 [00:00<?, ?ba/s]

100%|██████████| 125/125 [04:53<00:00,  2.35s/it]

['', '', 'Secretary']


  0%|          | 0/2 [00:00<?, ?ba/s]

 40%|████      | 50/125 [01:57<02:53,  2.32s/it]

In [ ]:
df[['title', 'post_concepts2', 'post_counters']].head()

In [ ]:
conclusions = [[x[1] for x in row] for row in df.post_counters.tolist()]
counters = [[x[2] for x in row] for row in df.post_counters.tolist()]

In [ ]:
best_conclusions, best_counters = zip(*get_best_counters(conclusions, counters))

In [ ]:
df['pred_counter_arguments_no_stance'] = best_counters
df['pred_conclusions_no_stance'] = best_conclusions

In [ ]:
df[['title', 'post_counters', 'pred_conclusions_no_stance', 'pred_counter_arguments_no_stance']].head()

In [ ]:
df.to_pickle('../multitask-counter-arg-generation/data/output/ca-final-models/mt-v4/results/test_prompts_preds_df.pkl')

#### Predict single counters:

In [16]:
conclusion_gen_kwargs = {
    "do_sample": True, 
    "max_length":50,
    "top_p":0.95, 
    "num_beams":1,
}

argument_gen_kwargs = {
    "do_sample": True, 
    "max_length":100,
    "min_length":50,
    "top_p":0.95, 
    "no_repeat_ngram_size":3,
    "top_k": 50,
    "num_beams":4,
}

In [18]:
ds = Dataset.from_pandas(df[['post']])
ds = ds.map(lambda a: tokenizer(a['post'], padding='max_length', max_length=256, truncation=True), 
                                   remove_columns=[ '__index_level_0__'], batched=True)

ds.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(ds, batch_size=batch_size)

Parameter 'function'=<function <lambda> at 0x7fe4dc16c040> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

In [22]:
no_stance_conclusions, no_stance_counter_arguments = generate_counters(model_without_stance, tokenizer, dataloader, argument_gen_kwargs, conclusion_gen_kwargs)

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:2259: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


In [23]:
df['single_pred_counter_arguments_no_stance'] = no_stance_counter_arguments
df['single_pred_conclusion_no_stance'] = no_stance_conclusions

In [24]:
df[['title', 'single_pred_counter_arguments_no_stance' , 'pred_counter_arguments_no_stance']].head()

,title,single_pred_counter_arguments_no_stance,pred_counter_arguments_no_stance
500025,I Don't Believe in the Big Bang,"the big bang is just a scientific theory. we have no way to prove or disprove the existence of the big bang. it's just a theory. if we accept it as a fact, then we have to accept that it is true. the only way to disprove it is to find a way to test it.","the big bang was never a fact, its an idea, and science can't prove it; it just means the universe itself wasn't able to reproduce in a consistent way. it is a scientific hypothesis, not a solid one at least. it has all the components of science, yet still can't be proven."
534021,There is no reason for Britain to remain in the EU.,"the eu is a single market. britain has a single currency, and british citizens have the right to travel to european countries, and europeans have a right to come to the eu to do the same. it is not a free trade zone, it is a customs union.","Idea for british independence is to give up everything in order to build a country that people want and use to their advantage. they are entitled to those ideals, even if they are not in fact what they want. a huge part of that freedom of movement is freedom not allowing people from another country to visit each other or from others to the rest of the world to contact each other."
415645,The Trump Administration Would Be Significantly More Harmful to US Domestic and International Policy W/out Moderates Like Mattis,"i don't think mattis is the only one who should be held accountable for the actions of the president. mattis has done a lot of good things for the country, but i don't see him being held accountable. the president is responsible for his actions, but he can't be held responsible for every decision he makes. for example, the decision to ban transgender people from entering the military did not come from mattis, it came from the president, and mattis","Surgery on transgender people is an incredibly dangerous decision. let's face it, surgery has an already high risk of suicide even after all the precautions, even if there's no risk to you, at best the potential for death and serious physical consequences is pretty high. the other risk is a potential war against a country without a president if the people make a choice to not serve or refuse to serve. for a lot of people, the only choice between hating the person or supporting him is"
507298,Poisoning the well against fascists is a dishonest and therefore ineffective way of combating fascists,"i think you're missing the point of fascism in the first place. it's a term that is used to describe a collection of people who want to impose their views on the rest of the world. fascism is not a movement, it's an ideology. a fascist is a person who wants to be a part of a group of people that wants to impose those ideas on the world, and a fascist who is opposed to that group because they don't want to be part of that","Fascism, at its best, can be described as a belief system defined with an explicit, objective, unambiguous way to judge, and even an idea itself. to describe it as being a totalitarian state or a fascist regime is to confuse one's self, as the movement can be viewed as a symptom of an underlying cause. fascism has always been the driving force of racism. fascism is the reason that black people were forced to choose between white and minority."
505763,Women have an easier time finding romantic partners than men,"i don't think that women have a harder time finding a romantic partner than men. i think it's just a matter of how you look at it, and how you approach it. women are much more likely to date incels than men, and i think that is a result of societal pressures, rather than a lack of a desire to date at all. i'm not saying that women are better at finding romantic partners than men are, but i do think that they","Industry (economics) is all about economics in the first place. if the rich could buy off any man as a'man', socie

In [26]:
df.to_pickle('../multitask-counter-arg-generation/data/output/ca-final-models/mt-v4/results/test_prompts_preds_df.pkl')

----------

In [10]:
df = pd.read_pickle('../multitask-counter-arg-generation/data/output/ca-final-models/mt-v4/results/test_prompts_preds_df.pkl')